# Assignment for Topic 5

## Part 1

Analyse the differences between the sexes by age in Ireland (not regions)

Using [CSO data](https://data.cso.ie/), load data from the FY006A - Population database. 

### Import the csv data from the url containing the data

In [ ]:
# import pandas

import pandas as pd


In [ ]:
# Define url
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"

# Define dataframe and import data from url 
df = pd.read_csv(url)

# Sanity check, show first 5 rows (shows "all sexes"), then the last 5 rows ("shows ")
df.head(-5)

# check this against csv I downloaded from the CSO (to be sure)

# Also, send to csv to check all of the data
df.to_csv("population_import.csv")

# After checking the population_import.csv file against the downloaded CSV file (which is in Excel), 
# I am happy that the import has worked correctly.


Observation: I only want male and female for sex comparison so I want to get rid of "Both sexes"

### Remove "Both sexes"

In [ ]:
# Remove "Both sexes" from the "Sex" column"
df = df[df["Sex"] != "Both sexes"]

# Sanity check here (commented out when I am happy with it, also sending to csv below will void this output)
# df.head(5)

# As it's first attempt at this task, I will do a second sanity check by again exporting to csv to check 
# the data
df.to_csv("population_male_female.csv")

# I am happy that the data now only contains male and female only in the "Sex" column

### Remove all ages as I don't need this

In [ ]:
# Remove "All ages" from the "Single Year of Age" column"
df = df[df["Single Year of Age"] != "All ages"]

# Sanity check here (commented out when I am happy with it)
# df.head(5)

# Again, I will do a second sanity check by again exporting to csv to check the data
df.to_csv("population_ages.csv")

# I am happy that the data now only contains no reference to "all ages", and male and female 
# only in the "Sex" column

### Remove columns I don't need

I can see that I don't want the following information: STATISTIC, statistic Label, TLIST(A1), CensusYear, C02199V02655, C02076V03371, C03789V04537, and UNIT. So I will remove them, leaving me with Index, Sex, Single Year of Age, Administrative Counties, VALUE.

In [ ]:
headers = df.columns.tolist()
headers

In [ ]:
# Use the headers list to create a list to remove

drop_col_list = ['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear', 'C02199V02655', 'C02076V03371', 'C03789V04537', 'UNIT']
df.drop(columns=drop_col_list, inplace=True)

# Sanity check to csv as warnings appearing so trying to see where the issue is. Works to the write
# to csv stage (commented out)
df.to_csv("population_columns_dropped.csv")

In [ ]:
# Remove text from ages
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

# Save to csv for sanity check
df.to_csv("population_clean_ages.csv")

### Note on above syntax warnings


In [ ]:
# Following on from my note below
# Remove "Ireland" from the "Administrative Counties" column"
df = df[df["Administrative Counties"] != "Ireland"]

# Sanity check here (commented out when I am happy with it)
#df.head(5)

# Again, I will do a second sanity check by again exporting to csv to check the data
df.to_csv("population_no_Ireland.csv")

In [ ]:
# Define values

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')

# Look at the dataframe
print (df.head(3))
df.info()

In [ ]:
# Use a pivot table to reframe the dataframe for analysis
# Investigating why my pivot table is giving warnings:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html
# Conversation with chatgpt: 

df_analysis = pd.pivot_table(df, values = 'VALUE', index = ['Administrative Counties', 'Single Year of Age'], columns= 'Sex', aggfunc="sum")
print (df_analysis)

# Save the csv ready for analysis
df.to_csv("population_for_anal.csv")

## Analysis of the population_for_anal.csv file

### First, determine the weighted mean age (by sex)

In [ ]:
# Define where each of the sexes live

headers = list(df_analysis.columns)
female = headers[0]
male = headers[1]
female, male

In [ ]:
# Weighted mean age calculation
number_female = df_analysis[female].sum()
number_female

In [ ]:
# Weighted mean age calculation
number_male = df_analysis[male].sum()
number_male

Note: at this stage both male and females are approx 5million, which i know is approx. the population of the country. So I think i forgot to remove "Ireland" counts for each sex, and age. Will go back and remove that and hopefully, this comment will reflect my thinking but above will actually be correct!
I think this had worked as the amount of each is about 2.6million now. 

In [ ]:
df_analysis 

In [ ]:
# Following your notebook, I will use the numpy method to calculate the weighted mean

import numpy as np

# Weighted mean age female
# AI suggested this code, makes sense to me
w_mean_female = np.average(df_analysis.index.get_level_values('Single Year of Age'), weights=df_analysis[female])
w_mean_female

In [ ]:
# Weighted mean age male
w_mean_male = np.average(df_analysis.index.get_level_values('Single Year of Age'), weights=df_analysis[male])
w_mean_male

In [ ]:
# The differences between the sexes
# minus the smaller from the larger and round to 2 decimal places

difference = w_mean_female - w_mean_male
round_difference = round(difference, 2)
round_female_w = round(w_mean_female, 4)
round_male_w = round(w_mean_male, 4)

print(f"The difference between the weighted mean age of the sexes in Ireland (to two decimal places) is {round_difference} years.\nTo four decimal places, the female weighted mean age is {round_female_w} years and the male weighted mean age is {round_male_w} years.")

# Part 2

## Create a variable that stores an age (40). Group the people within 5 years of that age together,
## into one age group. Calculate the population differences between the sexes in that age group.

### Come up with a plan of what I would like to try:

1. Store an age (40 years old)

2. Take only the Ireland data for each age, no administrative county. I will have to go back and see which csv i want to work on. Use this to remove all admin counties that aren't ireland and export a csv with only ireland data for each single age year

3. define the age 40 by where it is in the index/column in this df

4. define a function to group ages 35-45

5. calculate the difference between male and female for this age group
        

In [ ]:
# Create a df from the csv i want to conduct this analysis on, to make sure i'm dealing with the correct data

df_age = pd.read_csv("population_for_anal.csv")

# Sanity check
df_age.tail(10)

In [ ]:
# From conversation with chatgpt

# Age to extract
age_to_extract = 40

# Filter for that age across all counties
age_df = df_age[df_age["Single Year of Age"] == age_to_extract]

print(age_df)

# Export to csv to sanity check
age_df.to_csv("population_40.csv")


### To group
Try [pd.df.groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function

In [ ]:
# AFilter for ages 35-45
df_age_group = df[(df["Single Year of Age"] >= 35) & (df["Single Year of Age"] <= 45)].copy()

# Create a single age_group cplumn
df_age_group['age_group'] = '35-45'

# Aggregate by county, sex and age_group
grouped = df_age_group.groupby(["Administrative Counties", "Sex", "age_group"], as_index=False)["VALUE"].sum()
print(grouped)

# Export to csv to sanity check
grouped.to_csv("population_35-45.csv")

# Checked outputs against numbers in population_for_anal.csv and all looks good

In [ ]:
# Filter for males, sum the values across all counties for the 35-45 year old age group

# Filter for males in the 35-45 age group
males_35_45 = df_age_group[df_age_group['Sex'] == 'Male']

# Cumulative sum across all counties
total_males_35_45 = males_35_45['VALUE'].sum()

print(total_males_35_45)

In [ ]:
# Filter for females, sum the values across all counties for the 35-45 year old age group

# Filter for males in the 35-45 age group
females_35_45 = df_age_group[df_age_group['Sex'] == 'Female']

# Cumulative sum across all counties
total_females_35_45 = females_35_45['VALUE'].sum()

print(total_females_35_45)

In [ ]:
# The population difference between the sexes in that age group
difference = (total_females_35_45 - total_males_35_45)
print(f'The population difference between males and females in the age range 35-45 years is {difference}.\nThe total number of males is {total_males_35_45} and the total number of females is {total_females_35_45}.')
    

Having done all of that, I've just remembered there is an "Ireland" region in the df so I could have made life a lot easier for myself, taking this region instead of working at the county level and summing them up for a population level. None-the-less, there was probably more learning this way so I haven't re-done it!

# Part 3


## Which region in Ireland has the biggest population difference between the sexes in the 35-45 year old age group


Plan for part 3:

1. Define the amount of people in the 35-45 year old age group for each administrative county (above)

2. For each county, minus population males, population females from the other, and find the absolute value of this 

4. Rank all of these value for each county in order, or perhaps compile them all as a list and find the maximum value (would be quicker if it worked)

5. Find the highest/maximum (as applicable, depending on my method) value and go back to see which county this came from (i.e. look at the index for it)

In [ ]:
# Now, I am glad I did it all county by county basis as I have the values for each county of the
# 35-45 year old age group above

# From chatgpt, I should assign male and female values separate columns for each county, using a pivot table
# Then subtract these. I will save the difference to a new column

separate_sex = grouped.pivot(index = 'Administrative Counties', columns = 'Sex', values = 'VALUE')

# Sanity check
print(separate_sex)

Sex                                    Female   Male
Administrative Counties                             
Carlow County Council                    5338   5044
Cavan County Council                     6922   6513
Clare County Council                    10470   9488
Cork City Council                       19208  18488
Cork County Council                     31479  28906
Donegal County Council                  13256  12213
Dublin City Council                     52363  53401
Dún Laoghaire Rathdown County Council   19431  17888
Fingal County Council                   32930  30323
Galway City Council                      7427   7093
Galway County Council                   16524  15216
Kerry County Council                    12884  11827
Kildare County Council                  23188  21741
Kilkenny County Council                  8675   8112
Laois County Council                     8466   8025
Leitrim County Council                   2803   2658
Limerick City & County Council          17584 

In [ ]:
# Find the difference between male and female for each county and save to new column "female_minus_male"

separate_sex['female_minus_male'] = separate_sex['Female'] - separate_sex['Male']

print(separate_sex)

# Save to csv for sanity check
separate_sex.to_csv("population_35-45_male_female.csv")

Sex                                    Female   Male  female_minus_male
Administrative Counties                                                
Carlow County Council                    5338   5044                294
Cavan County Council                     6922   6513                409
Clare County Council                    10470   9488                982
Cork City Council                       19208  18488                720
Cork County Council                     31479  28906               2573
Donegal County Council                  13256  12213               1043
Dublin City Council                     52363  53401              -1038
Dún Laoghaire Rathdown County Council   19431  17888               1543
Fingal County Council                   32930  30323               2607
Galway City Council                      7427   7093                334
Galway County Council                   16524  15216               1308
Kerry County Council                    12884  11827            

In [ ]:
# Find the maximum value i.e. the biggest difference

max_value = separate_sex['female_minus_male'].max()

NameError: name 'female_minus_male' is not defined